***importing packages and libraries***

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import RandomBrightness, RandomContrast, RandomZoom
import pathlib as pl
import xml.etree.ElementTree as ET
from configs import modelConfigs as c

In [ ]:
def pre_process_image(image_path, target_height):
    # Load image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1)  # Decode to grayscale
    
    # Convert image to float32 and normalize to [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    shape = tf.shape(image)

    # Convert to numpy array for shape
    h, w, _ = shape.numpy()   
    # h, w = shape[0], shape[1] #this line is probs works and is more computationally effiiecnient than the line above

    # Calculate new width based on target height
    aspect_ratio = tf.cast(w, tf.float32) / tf.cast(h, tf.float32)
    new_width = tf.cast(target_height * aspect_ratio, tf.int32)

    # Resize image
    image_resized = tf.image.resize(image, [target_height, new_width])

    return image_resized

In [ ]:
# a function to seperate the forms iamges computer text written parts from the hand written parts
# important as label would need to be doubled to train both parts and other training complications
# the name portion is also needs to be removed as their is not training data on the text
def forms_text_seporator(form_path, bounding_box):
    image = tf.io.read_file(form_path)
    # handwritten part
    HW_part = tf.image.crop_and_resize([image], bounding_box, [0], ...)
    # computer text part
    CT_part = tf.image.crop_and_resize([image], bounding_box, [0], ...)
    
    return CT_part, HW_part 

In [ ]:
# TODO add more severe augmentations
# TODO build a seperate augmentation model to add padding for cropped photos or incoroporate into this model
def build_augmentation_model():
    augmentation_model = tf.keras.Sequential([
        RandomBrightness(factor=(-0.1, 0.1), value_range=(0, 1)),
        RandomContrast(factor=0.05),
        RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode='constant', fill_value=1.0)     
    ])
    return augmentation_model


In [ ]:
# TODO check if this works 
def form_bounding_box_finder(form_path):
    ...

In [ ]:
def data_preparator(X_image_paths, Y_image_path , data_length =1000 , image_target_height = 512, augmentation_probability = 0.3, data_split = (0.4, 0.6)):
    # directory containing labels for training data
    label_dir = pl.Path(Y_image_path)
    # forms and lines paths
    forms_pat = X_image_paths[0]
    lines_path = X_image_paths[1]
    # training example and label data X and Y
    X = []
    Y = []
    # keep track of the number files are being added to the data batch
    counter = 0
    # data augmentor
    augmentation_model = build_augmentation_model()

    for XML_path in label_dir.iterdir():
        if counter == data_length:
            break
        # get XML root element 
        root = ET.parse(str(XML_path)).getroot()
        # a lines in the XML file
        all_line_ele = root.find('handwritten-part')
        lines = all_line_ele.findall('line')
        # get bounding boxes feor handwritten part 
        # bouding box in the convention [y1, x1, y2, x2]
        form_crop_bounding_box = [0] * 4

        line_counter = 0
        line_nums = len(lines) # the number of lines
        # sub foilder for form that contains the lines for that form
        subf_path = root.get('id')

        # for lines
        for line in lines:
            #TODO figure out how to make int sequence label
            _ = line.get('text')

            # image path in the subfolder
            image_subf_path = line.get('id')
            # sulber folder name (is the first 3 chars)
            subf_name = image_subf_path[:3]
            
            full_image_path = f'{lines_path}/{subf_name}/{subf_path}/{image_subf_path}'
            print(full_image_path)
            #process the image 
            image = pre_process_image(full_image_path, image_target_height)
            #randomly with a chosen probability augment
            if np.random.rand() < augmentation_probability:
                image = augmentation_model(image)
            X.append(image)
            counter += 1
            # find X coords of the words
            # TODO abstract his into a function to tidy up function 
            words = line.findall('word')
            for word in words:
                chars = word.finall('cmp')
                for char in chars:
                    # first x coord
                    x_val = int(char.get('x'))
                    if form_crop_bounding_box[1] == 0:
                        form_crop_bounding_box[1] == x_val
                    elif form_crop_bounding_box[1] > x_val:
                        form_crop_bounding_box[1] = x_val
                    #  second x coord 
                    if form_crop_bounding_box[3] == 0:
                        form_crop_bounding_box[3] == x_val
                    elif form_crop_bounding_box[3] < x_val:
                        form_crop_bounding_box[3] = x_val
                    # handling y coords cases
                    y_val = int(char.get('y'))
                    if line_counter == 0:
                        if form_crop_bounding_box[0] == 0:
                            form_crop_bounding_box[0] = y_val
                        elif form_crop_bounding_box[0] > y_val:
                            form_crop_bounding_box[0] = y_val
                    elif line_counter == line_nums - 1:
                        if form_crop_bounding_box[2] == 0:
                            form_crop_bounding_box[2] = y_val
                        elif form_crop_bounding_box[2] < y_val:
                            form_crop_bounding_box[2] = y_val

            line_counter += 1
            

        usefull_text_labe = root.find('machine-printed-part')
        com_string = ''
        for lines in usefull_text_labe.findall('machine-print-line'):
            #add string togeather for complete sequence (obviously then convert into sequence data)
            com_string += lines # TODO verfiy this works  
            print(com_string)
            print(form_crop_bounding_box)
        



    # return np.array(X), np.array(Y)



In [ ]:
data_preparator(c().image_paths, c().label_path, 1 )